In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/My Drive/hw3_rl')

In [ ]:
!cp /content/drive/My\ Drive/hw3_rl/gridworld_hw3_q1.py /content/
!cp /content/drive/My\ Drive/hw3_rl/gw55_hw3.py /content/
!cp /content/drive/My\ Drive/hw3_rl/gw55_hw3_q1.py /content/
!cp /content/drive/My\ Drive/hw3_rl/utils.py /content/

In [ ]:
from gridworld_hw3_q1 import GridWorld
import numpy as np
import os
import random

#From gw55_hw3
def GridWorld5x5(p=0.9):
    rewards = {
        (2,0): -100,
        (4,4):  100
    }
    walls = [(2,2), (2,3), (2,4), (3,2)]

    # deterministic transition table
    T = {
        (0,0): { 'R': (0,1), 'D': (1,0) },
        (0,1): { 'R': (0,2), 'L': (0,0), 'D': (1,1) },
        (0,2): { 'R': (0,3), 'L': (0,1), 'D': (1,2) },
        (0,3): { 'R': (0,4), 'L': (0,2), 'D': (1,3) },
        (0,4): { 'L': (0,3), 'D': (1,4) },

        (1,0): { 'R': (1,1), 'D': (2,0), 'U': (0,0) },
        (1,1): { 'R': (1,2), 'L': (1,0), 'D': (2,1), 'U': (0,1) },
        (1,2): { 'R': (1,3), 'L': (1,1), 'U': (0,2) },
        (1,3): { 'R': (1,4), 'L': (1,2), 'U': (0,3) },
        (1,4): { 'L': (1,3), 'U': (0,4) },

        (2,0): { 'R': (2,1), 'D': (3,0), 'U': (1,0) },
        (2,1): { 'L': (2,0), 'D': (3,1), 'U': (1,1) },

        (3,0): { 'R': (3,1), 'D': (4,0), 'U': (2,0) },
        (3,1): { 'L': (3,0), 'D': (4,1), 'U': (2,1) },
        (3,3): { 'R': (3,4), 'D': (4,3) },
        (3,4): { 'L': (3,3), 'D': (4,4) },

        (4,0): { 'R': (4,1), 'U': (3,0) },
        (4,1): { 'R': (4,2), 'L': (4,0), 'U': (3,1) },
        (4,2): { 'R': (4,3), 'L': (4,1) },
        (4,3): { 'R': (4,4), 'L': (4,2), 'U': (3,3) },
        (4,4): { 'L': (4,3), 'U': (3,4) },
    }

    for s in T:
        ns_l = list(T[s].values())
        for a in T[s]:
            ns = T[s][a] #next state
            rs = ns_l[np.random.choice(len(ns_l))] #random
            if ns == rs:
                T[s][a] = { ns: 1.0 }
            else:
                T[s][a] = { ns: p, rs: np.round(1-p,2) }

    g = GridWorld(5, 5, start_position=(0, 0),
            pass_through_reward=0, rewards=rewards, walls = walls)
    g.probs = T

    return g

In [ ]:
from gridworld_hw3_q1 import GridWorld
#from gw55_hw3 import *
import numpy as np
import os
import random

# reward shaping global constants
REWARD_BONUS = 4          # Bonus for moving closer to the goal
PENALTY = -1              # Penalty for moving further away
OSCILLATION_PENALTY = -6   # Penalty for oscillatory moves
GOAL_STATE = (4, 4)

def manhattan_distance(state, goal):
    #distance from current position to goal
    return abs(state[0] - goal[0]) + abs(state[1] - goal[1])

def get_states(g):
    augmented = []
    for s in g.all_states():
        # Initial augmented state when no previous state exists
        augmented.append((s, None))
        for s_prev in g.all_states():
            augmented.append((s, s_prev))
    return augmented

def value_iteration(g, gamma=0.9, theta=1e-4, max_iterations=1000):
    augmented_states = get_states(g)
    # Initialize value function
    V = {state: 0 for state in augmented_states}
    iteration = 0

    while True:
        delta = 0
        iteration += 1
        for (s, last) in augmented_states:
            # If s is terminal, skip update
            if s in g.rewards:
                continue

            best_value = float('-inf')
            possible_actions = g.actions(s)

            # V(s) equation for each action
            for a in possible_actions:
                value = 0
                next_states_probs = g.probs.get(s, {}).get(a, {})
                for s_next, prob in next_states_probs.items():

                    new_state = (s_next, s)
                    base_reward = g.world[s_next]

                    # Apply oscillation penalty if s_next equals the last state
                    extra_penalty = OSCILLATION_PENALTY if (last is not None and s_next == last) else 0
                    r = base_reward + extra_penalty
                    value += prob * (r + gamma * V.get(new_state, 0))
                best_value = max(best_value, value)
            delta = max(delta, abs(best_value - V[(s, last)]))
            V[(s, last)] = best_value
        if delta < theta or iteration >= max_iterations:
            break

    # Extract optimal policy based on augmented states and value function
    policy = {}
    for (s, last) in augmented_states:
        if s in g.rewards:
            continue      #Skip our terminal states
        best_action = None
        best_value = float('-inf')
        possible_actions = g.actions(s)
        for a in possible_actions:
            value = 0
            next_states_probs = g.probs.get(s, {}).get(a, {})

            # Applies the oscillation penalty to discourage staying in the same place
            for s_next, prob in next_states_probs.items():
                new_state = (s_next, s)
                base_reward = g.world[s_next]
                extra_penalty = OSCILLATION_PENALTY if (last is not None and s_next == last) else 0
                r = base_reward + extra_penalty
                value += prob * (r + gamma * V.get(new_state, 0))
            if value > best_value:
                best_value = value
                best_action = a
        policy[(s, last)] = best_action

    return policy, V

In [ ]:
if __name__ == '__main__':
    g = GridWorld5x5()
    policy, V = value_iteration(g,gamma=0.9)

    # Initialize augmented state: starting at g.start_position with no last state
    current_augmented_state = (g.start_position, None)

    while not g.game_over():
        g.print()
        print()

        current_state, last_state = current_augmented_state
        actions = g.actions(current_state)
        # Choose action from policy for the augmented state
        action = policy.get(current_augmented_state, random.choice(actions))
        print(f"Chosen action: {action}")

        next_state, reward = g.move(action)

        # Compute Manhattan distances for reward shaping
        md_current = manhattan_distance(current_state, GOAL_STATE)
        md_next = manhattan_distance(next_state, GOAL_STATE)
        if md_next < md_current:
            reward += REWARD_BONUS
            print(f"Reward bonus for moving closer: {REWARD_BONUS}")
        elif md_next > md_current:
            reward += PENALTY
            print(f"Penalty for moving further: {PENALTY}")


        print(f"Moved to state: {next_state}")
        print(f"Reward: {reward}")

        # Update the agent's state
        g.set_state(next_state)
        current_augmented_state = (next_state, current_state)

    # Print the optimal augmented policy and value function
    print("Optimal Policy:")
    for r in range(g.rows):
        for c in range(g.columns):
            key = ((r, c), None)
            if key in policy:
                print(f"{(r, c)}: {policy[key]}", end=", ")
        print()

    print("Final Value Function:")
    for r in range(g.rows):
        for c in range(g.columns):
            key = ((r, c), None)
            print(f"{(r, c)}: {V.get(key, 0):.2f}", end=", ")
        print()

+-+-+-+-+-+
|o| | | | |
+-+-+-+-+-+
| | | | | |
+-+-+-+-+-+
|B| |x|x|x|
+-+-+-+-+-+
| | |x| | |
+-+-+-+-+-+
| | | | |G|
+-+-+-+-+-+

Chosen action: R
Reward bonus for moving closer: 4
Moved to state: (0, 1)
Reward: 4.0
+-+-+-+-+-+
| |o| | | |
+-+-+-+-+-+
| | | | | |
+-+-+-+-+-+
|B| |x|x|x|
+-+-+-+-+-+
| | |x| | |
+-+-+-+-+-+
| | | | |G|
+-+-+-+-+-+

Chosen action: D
Reward bonus for moving closer: 4
Moved to state: (1, 1)
Reward: 4.0
+-+-+-+-+-+
| | | | | |
+-+-+-+-+-+
| |o| | | |
+-+-+-+-+-+
|B| |x|x|x|
+-+-+-+-+-+
| | |x| | |
+-+-+-+-+-+
| | | | |G|
+-+-+-+-+-+

Chosen action: D
Reward bonus for moving closer: 4
Moved to state: (2, 1)
Reward: 4.0
+-+-+-+-+-+
| | | | | |
+-+-+-+-+-+
| | | | | |
+-+-+-+-+-+
|B|o|x|x|x|
+-+-+-+-+-+
| | |x| | |
+-+-+-+-+-+
| | | | |G|
+-+-+-+-+-+

Chosen action: D
Reward bonus for moving closer: 4
Moved to state: (3, 1)
Reward: 4.0
+-+-+-+-+-+
| | | | | |
+-+-+-+-+-+
| | | | | |
+-+-+-+-+-+
|B| |x|x|x|
+-+-+-+-+-+
| |o|x| | |
+-+-+-+-+-+
| | | | |G|
+-+-

In [ ]:
if __name__ == '__main__':
    g = GridWorld5x5(p=0.9)
    policy, V = value_iteration(g, gamma=0.3)

    # Initialize augmented state: starting at g.start_position with no last state
    current_augmented_state = (g.start_position, None)

    while not g.game_over():
        g.print()
        print()

        current_state, last_state = current_augmented_state
        actions = g.actions(current_state)
        # Choose action from policy for the augmented state
        action = policy.get(current_augmented_state, random.choice(actions))
        print(f"Chosen action: {action}")

        next_state, reward = g.move(action)

        # Compute Manhattan distances for reward shaping
        md_current = manhattan_distance(current_state, GOAL_STATE)
        md_next = manhattan_distance(next_state, GOAL_STATE)
        if md_next < md_current:
            reward += REWARD_BONUS
            print(f"Reward bonus for moving closer: {REWARD_BONUS}")
        elif md_next > md_current:
            reward += PENALTY
            print(f"Penalty for moving further: {PENALTY}")

        # (Oscillation penalty is already integrated in value_iteration)

        print(f"Moved to state: {next_state}")
        print(f"Reward: {reward}")

        # Update the agent's state: new augmented state becomes (next_state, current_state)
        g.set_state(next_state)
        current_augmented_state = (next_state, current_state)

    # Optionally print the optimal augmented policy and value function
    print("Optimal Policy:")
    for r in range(g.rows):
        for c in range(g.columns):
            key = ((r, c), None)
            if key in policy:
                print(f"{(r, c)}: {policy[key]}", end=", ")
        print()

    print("Final Value Function:")
    for r in range(g.rows):
        for c in range(g.columns):
            key = ((r, c), None)
            print(f"{(r, c)}: {V.get(key, 0):.2f}", end=", ")
        print()


+-+-+-+-+-+
|o| | | | |
+-+-+-+-+-+
| | | | | |
+-+-+-+-+-+
|B| |x|x|x|
+-+-+-+-+-+
| | |x| | |
+-+-+-+-+-+
| | | | |G|
+-+-+-+-+-+

Chosen action: D
Reward bonus for moving closer: 4
Moved to state: (1, 0)
Reward: 4.0
+-+-+-+-+-+
| | | | | |
+-+-+-+-+-+
|o| | | | |
+-+-+-+-+-+
|B| |x|x|x|
+-+-+-+-+-+
| | |x| | |
+-+-+-+-+-+
| | | | |G|
+-+-+-+-+-+

Chosen action: R
Reward bonus for moving closer: 4
Moved to state: (1, 1)
Reward: 4.0
+-+-+-+-+-+
| | | | | |
+-+-+-+-+-+
| |o| | | |
+-+-+-+-+-+
|B| |x|x|x|
+-+-+-+-+-+
| | |x| | |
+-+-+-+-+-+
| | | | |G|
+-+-+-+-+-+

Chosen action: D
Reward bonus for moving closer: 4
Moved to state: (2, 1)
Reward: 4.0
+-+-+-+-+-+
| | | | | |
+-+-+-+-+-+
| | | | | |
+-+-+-+-+-+
|B|o|x|x|x|
+-+-+-+-+-+
| | |x| | |
+-+-+-+-+-+
| | | | |G|
+-+-+-+-+-+

Chosen action: D
Reward bonus for moving closer: 4
Moved to state: (3, 1)
Reward: 4.0
+-+-+-+-+-+
| | | | | |
+-+-+-+-+-+
| | | | | |
+-+-+-+-+-+
|B| |x|x|x|
+-+-+-+-+-+
| |o|x| | |
+-+-+-+-+-+
| | | | |G|
+-+-